In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# AI Platform (Unified) Model Builder SDK: AutoML Forecasting Model Training Example

To use this Colaboratory notebook, you copy the notebook to your own Google Drive and open it with Colaboratory (or Colab). You can run each step, or cell, and see its results. To run a cell, use Shift+Enter. Colab automatically displays the return value of the last line in each cell. For more information about running notebooks in Colab, see the [Colab welcome page](https://colab.research.google.com/notebooks/welcome.ipynb).

This notebook demonstrates how to create an AutoML model based on a time series dataset. It will require you provide a bucket where the dataset will be stored.

Note: you may incur charges for training, prediction, storage or usage of other GCP products in connection with testing this SDK.

# Authenticate and Install Model Builder SDK

This section will authenticate and setup your environment to use an experimental version of the Model Builder SDK that contains support for AutoML Forecasting. Refer to the [Model Builder SDK User Guide](https://docs.google.com/document/d/1tFhzwCbR1jU-_BLkxmEriA9Y3NDWse2YYEYyKXkRybk) for an overview and detailed documentation can be downloaded from [here](https://storage.cloud.google.com/python-aiplatform/forecasting/v0.1/docs-0.6.0.zip).

After the SDK installation the kernel will be automatically restarted. You may see this error message `Your session crashed for an unknown reason` which is normal.

In [15]:
%%capture
%env SDK_VERSION=0.6.0

In [16]:
import sys
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()

In [1]:
%%capture
# hides output
!gsutil cp gs://python-aiplatform/forecasting/v0.1/google-cloud-aiplatform-${SDK_VERSION}.tar.gz .
!pip3 install google-cloud-aiplatform-${SDK_VERSION}.tar.gz
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Enter your project and GCS bucket

Enter your Project Id in the cell below. Then run the cell to make sure the Cloud SDK uses the right project for all the commands in this notebook.

In [1]:
MY_PROJECT = 'pytorch-tpu-nfs'

In [2]:
MY_STAGING_BUCKET = 'gs://automl-samples'  # bucket should be in same region as AI Platform (Unified)

The dataset we are using is the Rossman forecasting dataset.

# Initialize Model Builder SDK

Authenticate and initialize the *client* for AI Platform (Unified)

In [3]:
import sys
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()

In [4]:
from google.cloud import aiplatform

aiplatform.init(project=MY_PROJECT, staging_bucket=MY_STAGING_BUCKET)

# Create a Managed Time Series Dataset from CSV

This section will create a dataset from a CSV file stored on your GCS bucket

In [5]:
ds = aiplatform.datasets.TimeSeriesDataset.create(
    display_name='cmd-exp',
    bq_source=['bq://pytorch-tpu-nfs.view_dataset.weather_time_series_named'])

INFO:google.cloud.aiplatform.datasets.dataset:Creating TimeSeriesDataset
INFO:google.cloud.aiplatform.datasets.dataset:Create TimeSeriesDataset backing LRO: projects/64701051322/locations/us-central1/datasets/8629820475809202176/operations/5957488525703118848
INFO:google.cloud.aiplatform.datasets.dataset:TimeSeriesDataset created. Resource name: projects/64701051322/locations/us-central1/datasets/8629820475809202176
INFO:google.cloud.aiplatform.datasets.dataset:To use this TimeSeriesDataset in another session:
INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.TimeSeriesDataset('projects/64701051322/locations/us-central1/datasets/8629820475809202176')


# Launch a Training Job to Create a Model

Once we have defined your training script, we will create a model.

In [6]:
a='Date_Time, p__mbar, T__degC, Tpot__K, Tdew__degC, rh__percent, VPmax__mbar, VPact__mbar, VPdef__mbar, sh__g_per_kg, H2OC__mmol_per_mol, rho__gm_per_cubic_m, wv__m_per_s, max_w__vm_per_s, wd__deg'.split()
a

['Date_Time,',
 'p__mbar,',
 'T__degC,',
 'Tpot__K,',
 'Tdew__degC,',
 'rh__percent,',
 'VPmax__mbar,',
 'VPact__mbar,',
 'VPdef__mbar,',
 'sh__g_per_kg,',
 'H2OC__mmol_per_mol,',
 'rho__gm_per_cubic_m,',
 'wv__m_per_s,',
 'max_w__vm_per_s,',
 'wd__deg']

In [8]:
job = aiplatform.AutoMLForecastingTrainingJob(
    display_name='cmd-exp-job',
    optimization_objective='minimize-mae',    
    column_transformations=[
         {'timestamp': {'column_name': 'Date_Time'}},
         {'numeric': {'column_name': 'p__mbar'}},
         {'numeric': {'column_name': 'T__degC'}},
         {'numeric': {'column_name': 'Tpot__K'}},
         {'numeric': {'column_name': 'Tdew__degC'}},
         {'numeric': {'column_name': 'rh__percent'}},
         {'numeric': {'column_name': 'VPmax__mbar'}},
         {'numeric': {'column_name': 'VPact__mbar'}},
         {'numeric': {'column_name': 'VPdef__mbar'}},
         {'numeric': {'column_name': 'sh__g_per_kg'}},
         {'numeric': {'column_name': 'H2OC__mmol_per_mol'}},
         {'numeric': {'column_name': 'rho__gm_per_cubic_m'}},
         {'numeric': {'column_name': 'wv__m_per_s'}},
         {'numeric': {'column_name': 'max_w__vm_per_s'}},
         {'numeric': {'column_name': 'wd__deg'}}
    ]
)

features_col = [
    'Date_Time',
 'p__mbar',
 'Tpot__K',
 'Tdew__degC',
 'rh__percent',
 'VPmax__mbar',
 'VPact__mbar',
 'VPdef__mbar',
 'sh__g_per_kg',
 'H2OC__mmol_per_mol',
 'rho__gm_per_cubic_m',
 'wv__m_per_s',
 'max_w__vm_per_s',
 'wd__deg']

# This will take around an hour to run
model = job.run(
    dataset=ds,
    target_column='T__degC',
    time_column='Date_Time',
    time_series_identifier_column='Group',
    available_at_forecast_columns=features_col,
    unavailable_at_forecast_columns=['T__degC'],
    time_series_attribute_columns=[],
    forecast_horizon=24.0,
    data_granularity_unit='hour',
    data_granularity_count=1,
    weight_column=None,
    budget_milli_node_hours=1000,
    model_display_name='cmd-exp-model', 
    predefined_split_column_name=None,
    context_window=120,
)

INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/8862806989734936576?project=64701051322
INFO:google.cloud.aiplatform.training_jobs:AutoMLForecastingTrainingJob projects/64701051322/locations/us-central1/trainingPipelines/8862806989734936576 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLForecastingTrainingJob projects/64701051322/locations/us-central1/trainingPipelines/8862806989734936576 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLForecastingTrainingJob projects/64701051322/locations/us-central1/trainingPipelines/8862806989734936576 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLForecastingTrainingJob projects/64701051322/locations/us-central1/trainingPipelines/8862806989734936576 current state:
PipelineState.PIPELINE_STATE_RUNNING


RuntimeError: Training failed with:
code: 3
message: "Column \'p__mbar\' from transformation doesn\'t exist."


# Fetch Model Evaluation Metrics

In [ ]:
import pandas as pd

list_evaluation_pager = model.api_client.list_model_evaluations(parent=model.resource_name)
for model_evaluation in list_evaluation_pager:
  metrics_dict = {m[0]: m[1] for m in model_evaluation.metrics.items()}
  df = pd.DataFrame(metrics_dict.items(), columns=['Metric', 'Value'])
  print(df.to_string(index=False))

# Batch Prediction


In [13]:
MY_STAGING_BUCKET='gs://automl-samples'

In [14]:
BATCH_PREDICT_SOURCE = 'bq://pytorch-tpu-nfs.forecasting_exp.weather_test_set'
BATCH_PREDICT_DESTINATION_PREFIX = f'{MY_STAGING_BUCKET}/prediction'
model.batch_predict(
   bq_source=BATCH_PREDICT_SOURCE,
   job_display_name='predict-weather-time-series')

INFO:google.cloud.aiplatform.jobs:Creating BatchPredictionJob
INFO:google.cloud.aiplatform.jobs:BatchPredictionJob created. Resource name: projects/64701051322/locations/us-central1/batchPredictionJobs/7695517067176312832
INFO:google.cloud.aiplatform.jobs:To use this BatchPredictionJob in another session:
INFO:google.cloud.aiplatform.jobs:bpj = aiplatform.BatchPredictionJob('projects/64701051322/locations/us-central1/batchPredictionJobs/7695517067176312832')
INFO:google.cloud.aiplatform.jobs:View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/7695517067176312832?project=64701051322
INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/64701051322/locations/us-central1/batchPredictionJobs/7695517067176312832 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:BatchPredictionJob  run. Resource name: projects/64701051322/locations/us-central1/batchPredictionJobs/7695517067176312832


resource name: projects/64701051322/locations/us-central1/batchPredictionJobs/7695517067176312832